## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [107]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [108]:
import yaml

In [109]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [110]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [111]:
saut="\n"

### Préparation des matrices de traits

In [112]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [113]:
sampleNumber="derivation2-SILVER"
genDigraphe=False
genGraphe=False
#samplePrefix="MGC-150916-extend-%s-paradigmes"%sampleNumber
#samplePrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#samplePrefix="MGC-160104-%s-paradigmes"%sampleNumber
#samplePrefix="MGC-160104-basesDerives-%s"%sampleNumber
samplePrefix="MGC-160104-%s"%sampleNumber
sampleFile=samplePrefix+".csv"
#goldFile="MGC-160104-01-N-VerbesActions-GOLD.csv"
#goldFile="MGC-160104-01-N-Gold.csv"
#goldFile="MGC-160104-%s-GOLD.csv"%sampleNumber
#analysisPrefix="MGC-150916-extend-%s"%sampleNumber
#analysisPrefix="MGC-160104-%s-VerbesActions-SILVER"%sampleNumber
#analysisPrefix="MGC-160104-%s"%sampleNumber
analysisPrefix="MGC-160104-01-N-VerbesActions-SILVER"
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

### Préparation des cases du paradigme

In [114]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [115]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [116]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [117]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [118]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

# Classe pour la gestion des patrons, des classes et des transformations

In [119]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [120]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [121]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [122]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [123]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [124]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [125]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [126]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [127]:
paradigmes.stack().value_counts(dropna=True).sum()

4027

In [128]:
with open("2015-Data/"+analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [129]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(casesTotales):
    print ("Attention l'analyse ne comprend pas toutes les cases")

Attention l'analyse ne comprend pas toutes les cases


In [130]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [131]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [132]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [133]:
def generateParadigms(generation1,genDigraphe=True):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()    
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

In [134]:
def generate(lexeme,genDigraphe=True):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

In [135]:
len(paradigmes.dropna(thresh=2)["lexeme"])

150

In [136]:
paradigmes.dropna(thresh=2).count().sum()-paradigmes.dropna(thresh=2)["lexeme"].count()

3877

#### Calculer le score de la clique

In [137]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [138]:
#%%time
debug=False
#listeTest=[u"manger",u"boire",u"dormir",u"aller",u"neiger"]
#listeTest=paradigmes.dropna(thresh=2)["lexeme"].values.tolist()
listeTest=paradigmes.dropna(thresh=1)["lexeme"].values.tolist()
#listeTest=[u"abasourdir",u"évacuer"]
nbVerbes=len(listeTest)
print (nbVerbes)
globDigraphe=nx.DiGraph()
globGraphe=nx.Graph()
cliques=[]
cliquesScores={}
cliquesListes={}
numClique=0
progressBar = FloatProgress(min=0, max=nbVerbes)
display(progressBar)
for i,element in enumerate(listeTest):
#    if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
    progressBar.value=i
    result=generate(element,genDigraphe)
    (generation,lexDigraphe,lexGraphe,lexCliques)= result
#    print (generation,lexDigraphe,lexGraphe,lexCliques)
    if genDigraphe:
        globDigraphe=nx.union(globDigraphe,lexDigraphe)
    if genGraphe:
        globGraphe=nx.union(globGraphe,lexGraphe)
    cliques.extend(lexCliques)
    for clique in lexCliques:
        cliquesScores[numClique]=cliqueScore(clique,lexGraphe)
        cliquesListes[numClique]=clique
        numClique+=1

150


In [139]:
%%time
versionStamp=dateheure()
if genDigraphe: 
    nx.write_dot(globDigraphe,u"2015-Data/digraphe-%s.dot"%versionStamp)
if genGraphe:
    nx.write_dot(globGraphe,u"2015-Data/graphe-%s.dot"%versionStamp)

CPU times: user 39 µs, sys: 1 µs, total: 40 µs
Wall time: 44.1 µs


In [140]:
print (len(cliques))

35865


In [141]:
longueurCliques={}
for clique in cliques:
    longueur=len(clique)
    if not longueur in longueurCliques:
        longueurCliques[longueur]=1
    else:
        longueurCliques[longueur]+=1
longueurCliques

{1: 1,
 2: 411,
 3: 671,
 4: 757,
 5: 1176,
 6: 1500,
 7: 1907,
 8: 1612,
 9: 1607,
 10: 1741,
 11: 1679,
 12: 1467,
 13: 1520,
 14: 1729,
 15: 1320,
 16: 1210,
 17: 1254,
 18: 1678,
 19: 1386,
 20: 1825,
 21: 1754,
 22: 1598,
 23: 1268,
 24: 1209,
 25: 945,
 26: 620,
 27: 534,
 28: 400,
 29: 399,
 30: 283,
 31: 108,
 32: 80,
 33: 85,
 34: 58,
 35: 29,
 36: 20,
 37: 11,
 38: 5,
 39: 2,
 40: 2,
 41: 2,
 42: 1,
 44: 1}

### Faire la liste des cases lexicalisées de l'échantillon

In [142]:
nbFormesLexicales={}
casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
for element in casesLexicales:
    casesLexicales[element].remove("lexeme")
    nbFormesLexicales[element]=len(casesLexicales[element])

## Comparer la sortie des cliques avec le paradigme de départ

In [146]:
%%time
seuilClique=1
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
#for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
progressBar = FloatProgress(min=0, max=len(cliques))
display(progressBar)
for n,clique in enumerate(cliques):
    progressBar.value=n
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
        point=clique[0].split("-")
        lPoint=len(point)
        if lPoint==3:
            lexeme=point[0]
        else:
            lexeme="-".join(point[0:len(point)-2])
        paradigmeClique["lexeme"]=lexeme
        casesLexeme=casesLexicales[lexeme]
        nbInitial=len(casesLexeme)
        if casesLexeme and len(casesLexeme)<=len(clique):
            for element in casesLexeme:
                champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
                if ","  in champForme:
                    formes=champForme.split(",")
                    nbInitial+=len(formes)-1
                    okFormes=False
                    for forme in formes:
                        pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                        if pointCase in clique:
                            okFormes=True
                            if debug: print ("point",pointCase)
#                            if debug: print ("gold",GOLD[GOLD["lexeme"]==lexeme][element])
                    if okFormes:
                        sampleOK=True
                    else:
                        sampleOK=False
                        break
                else:
                    forme=champForme
                    pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                    if debug: print (pointCase, clique)
                    if not pointCase in clique:
                        sampleOK=False
                        break
        else:
            sampleOK=False
        if sampleOK:
            for element in clique:
                elements=element.split("-")
                forme=elements[-2]
                taminfo=elements[-1]
#                try:
#                 (lexeme,forme,taminfo)=element.split("-")
#                except ValueError:
#                    print (element)
                paradigmeClique[taminfo]=forme
#                if taminfo in paradigmes.columns:
#                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
#                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
#                            sampleOK=False
#                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
#            if n<100: print (clique==cliquesListes[n])
            paradigmeClique["score"]=cliquesScores[n]
            paradigmeClique["ajouts"]=len(clique)-nbInitial
#            print (cliqueScore(clique),clique)
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
#    else:
#        print ("break")
#        print (n,lexeme,clique)
#        break

#paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist()+[u"score",u"ajouts"])
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=[u"lexeme"]+sampleCases+[u"score",u"ajouts"])
#print (seuilClique)

CPU times: user 38.1 s, sys: 3.63 s, total: 41.8 s
Wall time: 41.8 s


In [147]:
paradigmesCLIQUES

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,FS,FP,score,ajouts
0,absoudre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,absudre,...,NaN,absud,NaN,NaN,NaN,absud,NaN,NaN,76.632158,0
1,accroître,NaN,NaN,NaN,NaN,NaN,NaN,NaN,akrwatrE,akrwatre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.498397,0
2,agonir,NaN,NaN,NaN,NaN,agOnir,agOni,agOnirô,agOnirE,agOnire,...,NaN,NaN,NaN,NaN,NaN,NaN,agOni,agOni,165.173767,0
3,aller,NaN,alE,NaN,NaN,alEr,ala,irô,irE,ire,...,NaN,aj,NaN,aj,aj,aj,ale,ale,813.394717,0
4,amener,NaN,NaN,NaN,NaN,am9nEr,am9na,NaN,amEn9rE,NaN,...,NaN,NaN,NaN,NaN,NaN,amEn,NaN,NaN,218.708361,0
5,astreindre,NaN,astrEJE,NaN,NaN,astrEJEr,astrEJa,NaN,astrEJ9rE,astrEJ9re,...,NaN,astrEJ,NaN,astrEJ,astrEJ,astrEJ,NaN,NaN,393.933058,0
6,attendre,NaN,atâdi,NaN,NaN,atâdir,atâdi,NaN,atâdrE,NaN,...,NaN,atâd,NaN,NaN,NaN,atâd,atât,atât,349.159657,0
7,buter,NaN,bytE,NaN,NaN,bytEr,byta,NaN,byt9rE,byt9re,...,NaN,byt,NaN,byt,NaN,byt,NaN,NaN,638.349553,0
8,chevaucher,NaN,S9vOSE,NaN,NaN,S9vOSEr,S9vOSa,NaN,S9vOS9rE,S9vOS9re,...,NaN,S9vOS,NaN,S9vOS,S9vOS,S9vOS,NaN,NaN,481.320714,0
9,conduire,NaN,NaN,NaN,NaN,NaN,kôdHizi,NaN,kôdHirE,NaN,...,NaN,kôdHiz,NaN,NaN,NaN,kôdHiz,kôdHit,kôdHit,174.878840,0


In [183]:
paradigmeMAX=paradigmesCLIQUES[paradigmesCLIQUES["score"]==paradigmesCLIQUES.groupby(["lexeme"])["score"].transform(max)].reset_index()
del paradigmeMAX["index"]
paradigmeMAX["score"]=paradigmeMAX["score"].apply(str)
paradigmeMAX["ajouts"]=paradigmeMAX["ajouts"].apply(str)
paradigmeSILVER=paradigmeMAX.groupby("lexeme").agg(lambda x: ",".join(list(set(x.dropna().values)))).reset_index()

In [185]:
paradigmeSILVER.to_csv(path_or_buf="2015-Data/"+sampleFile.replace(".csv","-Silver.csv"),encoding="utf8",sep=";")
#GOLD.to_csv(path_or_buf="2015-Data/"+analysisPrefix+'-Gold.csv',encoding="utf8",sep=";")

In [173]:
sampleFile.replace(".csv","-Silver.csv")

'MGC-160104-derivation2-SILVER-Silver.csv'

In [ ]:
#len(paradigmes.dropna(thresh=2)["lexeme"].values.tolist())

In [184]:
paradigmeSILVER

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,FS,FP,score,ajouts
0,absoudre,,,,,,,,,absudre,...,,absud,,,,absud,,,76.632157542,0
1,accroître,,,,,,,,akrwatrE,akrwatre,...,,,,,,,,,64.4983972437,0
2,agonir,,,,,agOnir,agOni,agOnirô,agOnirE,agOnire,...,,,,,,,agOni,agOni,165.173766817,0
3,aller,,alE,,,alEr,ala,irô,irE,ire,...,,aj,,aj,aj,aj,ale,ale,813.394716743,0
4,amener,,,,,am9nEr,am9na,,amEn9rE,,...,,,,,,amEn,,,218.708361351,0
5,astreindre,,astrEJE,,,astrEJEr,astrEJa,,astrEJ9rE,astrEJ9re,...,,astrEJ,,astrEJ,astrEJ,astrEJ,,,393.933057996,0
6,attendre,,atâdi,,,atâdir,atâdi,,atâdrE,,...,,atâd,,,,atâd,atât,atât,349.159657388,0
7,buter,,bytE,,,bytEr,byta,,byt9rE,byt9re,...,,byt,,byt,,byt,,,638.34955255,0
8,chevaucher,,S9vOSE,,,S9vOSEr,S9vOSa,,S9vOS9rE,S9vOS9re,...,,S9vOS,,S9vOS,S9vOS,S9vOS,,,481.320714029,0
9,conduire,,,,,,kôdHizi,,kôdHirE,,...,,kôdHiz,,,,kôdHiz,kôdHit,kôdHit,174.878839527,0


In [180]:
#paradigmeMAX.groupby("lexeme").agg(lambda x: ",".join(list(set(x.dropna().values)))).reset_index()
paradigmeMAX["score"].apply(str)
paradigmeMAX["ajouts"].apply(str)
paradigmeMAX

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,FS,FP,score,ajouts
0,absoudre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,absudre,...,NaN,absud,NaN,NaN,NaN,absud,NaN,NaN,76.632158,0
1,accroître,NaN,NaN,NaN,NaN,NaN,NaN,NaN,akrwatrE,akrwatre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.498397,0
2,agonir,NaN,NaN,NaN,NaN,agOnir,agOni,agOnirô,agOnirE,agOnire,...,NaN,NaN,NaN,NaN,NaN,NaN,agOni,agOni,165.173767,0
3,aller,NaN,alE,NaN,NaN,alEr,ala,irô,irE,ire,...,NaN,aj,NaN,aj,aj,aj,ale,ale,813.394717,0
4,amener,NaN,NaN,NaN,NaN,am9nEr,am9na,NaN,amEn9rE,NaN,...,NaN,NaN,NaN,NaN,NaN,amEn,NaN,NaN,218.708361,0
5,astreindre,NaN,astrEJE,NaN,NaN,astrEJEr,astrEJa,NaN,astrEJ9rE,astrEJ9re,...,NaN,astrEJ,NaN,astrEJ,astrEJ,astrEJ,NaN,NaN,393.933058,0
6,attendre,NaN,atâdi,NaN,NaN,atâdir,atâdi,NaN,atâdrE,NaN,...,NaN,atâd,NaN,NaN,NaN,atâd,atât,atât,349.159657,0
7,buter,NaN,bytE,NaN,NaN,bytEr,byta,NaN,byt9rE,byt9re,...,NaN,byt,NaN,byt,NaN,byt,NaN,NaN,638.349553,0
8,chevaucher,NaN,S9vOSE,NaN,NaN,S9vOSEr,S9vOSa,NaN,S9vOS9rE,S9vOS9re,...,NaN,S9vOS,NaN,S9vOS,S9vOS,S9vOS,NaN,NaN,481.320714,0
9,conduire,NaN,NaN,NaN,NaN,NaN,kôdHizi,NaN,kôdHirE,NaN,...,NaN,kôdHiz,NaN,NaN,NaN,kôdHiz,kôdHit,kôdHit,174.878840,0
